need a spline library with the right boundary conditions.


In [ ]:
using Dierckx #, PyPlot

In [ ]:
filename = "/Users/ortner/Downloads/ffe.plt.txt"
data = readdlm(filename, Float64)
x, y = data[:,1], data[:,2];

In [ ]:
# plot(x, y)
# axis([0.0, 6.0, -1.0, 2.0])

In [ ]:
spl = Spline1D(x, y; bc = "zero", s = 1e-2, w = (1 + abs(y)).^(-2))
spl.c[end-2:end] = 0.0
t = linspace(1.0, 6.0, 200) |> collect
# plot(t, spl(t), "b-", t, derivative(spl, t), "r-")
# axis([3, 6, -1e-2, 1e-2])


In [ ]:
# plot(x, y, "b-", x, spl(x), "r:")
# axis([0, 6, -1.0, 5.0])

In [ ]:
tt = 2.0 + rand(100_000);

In [ ]:
# Evaluations of the entire vector
@time spl(tt)
@time derivative(spl, tt)
@time derivative(spl, tt; nu=2)

# OLD 
#   0.006103 seconds (8 allocations: 781.594 KB)
#   0.003211 seconds (9 allocations: 781.781 KB)
#   0.002177 seconds (10 allocations: 781.875 KB)

# NEW
#   0.006344 seconds (7 allocations: 781.500 KB)
#   0.003319 seconds (8 allocations: 781.672 KB)
#   0.002090 seconds (9 allocations: 781.766 KB)
;

In [ ]:
function ftest(spl, tt)
    for t in tt
        spl(t) 
    end 
end

function ftest1(spl, tt)
    wrk = Vector{Float64}(length(spl.t))
    for t in tt
        Dierckx.__derivative(spl.t, spl.c, spl.k, t, 1, spl.bc, wrk)
        #         derivative(spl, t)
    end 
end

function ftest2(spl, tt)
    wrk = Vector{Float64}(length(spl.t))
    for t in tt
        Dierckx.__derivative(spl.t, spl.c, spl.k, t, 2, spl.bc, wrk)
        # derivative(spl, t, nu = 2)
    end 
end

In [ ]:
@time ftest(spl, tt)
@time ftest(spl, tt)
@time ftest1(spl, tt)
@time ftest1(spl, tt)
@time ftest2(spl, tt)
@time ftest2(spl, tt)

In [ ]:
# OLD TIMINGS 
# evaluate 
#   0.018588 seconds (500.00 k allocations: 15.259 MB, 11.57% gc time)
#   0.017436 seconds (500.00 k allocations: 15.259 MB, 7.48% gc time)
# first derivative
#   0.020274 seconds (600.00 k allocations: 33.569 MB, 6.01% gc time)
#   0.020318 seconds (600.00 k allocations: 33.569 MB, 11.01% gc time)
# second derivative
#   0.027993 seconds (700.00 k allocations: 42.725 MB, 11.26% gc time)
#   0.026323 seconds (700.00 k allocations: 42.725 MB, 10.23% gc time)

# NEW TIMINGS: REMOVE TYPE INSTABILITY
# evaluate 
#   0.009857 seconds (200.00 k allocations: 3.052 MB)
#   0.009378 seconds (200.00 k allocations: 3.052 MB)
# first derivative
#   0.012261 seconds (300.00 k allocations: 19.837 MB, 8.79% gc time)
#   0.012860 seconds (300.00 k allocations: 19.837 MB, 6.69% gc time)
# second derivative
#   0.019107 seconds (400.00 k allocations: 28.992 MB, 11.40% gc time)
#   0.018948 seconds (400.00 k allocations: 28.992 MB, 11.45% gc time)

# NEW TIMINGS: WORK ARRAY PASSED AS AN ARGUMENT 
# evaluate 
#   0.010958 seconds (200.00 k allocations: 3.052 MB)
#   0.009265 seconds (200.00 k allocations: 3.052 MB)
# first derivative
#   0.008837 seconds (200.00 k allocations: 3.052 MB)
#   0.009624 seconds (200.00 k allocations: 3.052 MB)
# second derivative
#   0.009792 seconds (200.00 k allocations: 3.052 MB)
#   0.009251 seconds (200.00 k allocations: 3.052 MB)

